In [15]:
from qanta.util import constants as c
from qanta.datasets.quiz_bowl import QuizBowlDataset
qbdb = QuizBowlDataset(1, buzzer_train=True)
questions = qbdb.questions_in_folds(['buzzerdev'])

In [60]:
protobowl_ids = {x.qnum: x.protobowl for x in questions}
print(len(protobowl_ids))

7387


In [30]:
def process_line(x):
    total_time = x['object']['time_elapsed'] + x['object']['time_remaining']
    ratio = x['object']['time_elapsed'] / total_time
    position = int(len(x['object']['question_text'].split()) * ratio)
    return [x['object']['guess'], x['object']['qid'], position, x['object']['ruling']]
data1 = list(map(process_line, data))

In [31]:
import sys
import json
import codecs
import pandas as pd

protobowl_dir = 'data/external/qanta-4may17.log'
data = []
data1 = []
count = 0
with codecs.open(protobowl_dir, 'r', 'utf-8') as f:
    line = f.readline().strip()
    while line is not None:
        while not line.endswith('}}'):
            line += f.readline().strip()
        try:
            line = json.loads(line)
        except ValueError:
            continue
        data.append(line)
        count += 1
        if count % 10000 == 0:
            print('\r\r{}'.format(count))
        data.append(line)
        data1.append(process_line(line))
        line = f.readline().strip()
print(len(data))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

KeyboardInterrupt: 

In [33]:
len(data1)

3603563

In [34]:
df = pd.DataFrame(data1, columns=['guess', 'qid', 'position', 'result'])

In [35]:
import pickle
pickle.dump(df, open('data/external/protobowl_df.pkl', 'wb'))

In [42]:
buzzes = pickle.load(open('output/buzzer/buzzes_buzzerdev.pkl', 'rb'))

In [45]:
buzzerdev_keys =[x.qnum for x in questions]
list(buzzes.keys())[0] in buzzerdev_keys

True

In [49]:
from qanta.guesser.abstract import AbstractGuesser
from qanta.buzzer import constants as bc
guesses = AbstractGuesser.load_guesses(bc.GUESSES_DIR, folds=['buzzerdev'])

In [55]:
guesses = guesses.groupby('qnum')
df = df.groupby('qid')

In [62]:
avg_len = 0
count = 0
for qnum, guesses_group in guesses:
    proto_id = protobowl_ids[qnum]
    if proto_id in df.groups:
        protos = df.get_group(proto_id)
        avg_len += len(protos)
        count += 1
print(avg_len / count)
print(count)

37.87687305626237
7074


In [65]:
from qanta.util.multiprocess import _multiprocess
from qanta.reporting.new_performance import _get_top_guesses
top_guesses = _multiprocess(_get_top_guesses, guesses)

[] done: 7386/7387


In [71]:
from qanta.buzzer.util import GUESSERS
n_guessers = len(GUESSERS)
print(n_guessers)

1


In [69]:
for qnum, guesses in top_guesses:
    if qnum not in protobowl_ids:
        continue
    protobowl_id = protobowl_ids[qnum]
    if protobowl_id not in df.groups:
        continue
    opponents = df.get_group(protobowl_id)
    buzz = buzzes[qnum]
    a